# Air Quality Over Time in the United States

## Introduction
The historical air quality dataset available on Kaggle by the USDA provides an opportunity to use machine learning algorithms to predict the air quality of a certain location based on its historical data. This notebook aims to explore the dataset and build machine learning models to predict the air quality of different locations.

In [ ]:
# Load packages
import numpy as np 
import pandas as pd
import os

# Big query helpers
from google.cloud import bigquery
from bq_helper import BigQueryHelper



import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Setup big data

In [ ]:
# Set up query helpers
bq_assistant = BigQueryHelper("bigquery-public-data", "epa_historical_air_quality")
pollutants = ['o3','co','no2','so2','pm25_frm']

## SQL queries

Now that we have set up the environment, we can write the SQL queries that will get the data we need. To make the choloropleth map, we need 1) the air pollution AQI summaries by county and 2) the FIPS or location codes of each state and county. 

In [ ]:
def get_pollutant_year_query(year):
    query = f"""
        SELECT
            pollutant.county_name AS county, AVG(pollutant.aqi) AS AvgAQI_pollutant,
            pollutant.state_code, pollutant.county_code
        FROM
          `bigquery-public-data.epa_historical_air_quality.pollutant_daily_summary` as pollutant
        WHERE
          pollutant.poc = 1
          AND EXTRACT(YEAR FROM pollutant.date_local) = {year}
        GROUP BY 
          pollutant.state_code, pollutant.county_code, pollutant.county_name
    """
    return query

## Data preparation 

We are now ready to run the SQL query.  We want to get the data from the daily summary table for each pollutant and each county. We could do the join in SQL using 'JOIN' commands but this will take a long time. Instead of doing this, we can run the queries for each pollutant and then merge the results in pandas. 

The merge in the code block below is an outer join because we want all the possible data that exists for each county. Just because a county does not have measurements for a particular pollutant does not mean that we want to discard all that county's data completely. We still want to retain whatever information exists. 

In [ ]:
# Initialize an empty dictionary to store dataframes for each year
dataframes_by_year = {}

In [ ]:
for year in range(2017, 2023):
    print(f"Processing data for year {year}...")
    try:
        # Initialize the data-frame for year
        df = pd.DataFrame()

        # Now loop through the pollutants list we already have
        for elem_g in pollutants : 

            # Replaces the word 'pollutant' in the query with the actual pollutant's name
            query = get_pollutant_year_query(year).replace("pollutant", elem_g)

            # Runs the query and transforms it to a pandas data-frame 
            # Create a joined up FIPS code that uniquely identifies counties
            # Set the index 
            temp = bq_assistant.query_to_pandas(query)
            temp['location_code'] = temp['state_code'] + temp['county_code']
            temp.set_index('location_code') 

            # Concatenate the tables of the different pollutants together 
            # Fill in the missing values with the mean of the column
            if elem_g == 'o3': 
                df = temp 

            # Merge on location code
            else:
                temp.drop(['state_code', 'county_code', 'county'], inplace = True, axis = 1)
                df = pd.merge(df, temp, how = 'outer', on = ['location_code'],
                                  indicator = elem_g + '_merge')
        
        # Add a new "year" column to the DataFrame
        df['year'] = year
        
        # add
        dataframes_by_year[year] = df
    except:
        print(year)
    

In [ ]:
dataframes_by_year[2022].head()

## Missing values 

The data we have seems fine. There is one catch: there are counties in the data-set where certain pollutants were not measured in year. For example, the Avg. AQI index value for Hendricks county is missing in the random sample that we have drawn above. We need to deal with these missing values in some way. The code block below fills in these missing values with the average value for that column.

In [ ]:
# Concatenate the data frames for all years
df_all = pd.concat(dataframes_by_year.values(), axis=0)

# Fill in the numeric missing values 
for column in df_all.columns: 
    if df_all[column].dtype in ['float64', 'int64']: 
        df_all[column].fillna(df_all[column].mean(), inplace = True)

# Randomly pick 10 counties to take a look at the data
df_all.sample(10, random_state=42)

## Creating the maps! 
The code below calls the Plotly API to create a choloropleth map for each pollutant at the county level for the USA. The high number of missing values in each plot are surprising. It would be interesting to predict these from the existing data.

In [ ]:
!pip install dash==1.20.0
!pip install -q plotly==3.6.1 


In [ ]:
# Import plotting libaries
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.io as pio

# Need this so we can use Plotly in offline mode
# This will allow the maps we make to show up in this notebook
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode()

def make_plot(pollutant, plot_labels, color_scale):
    '''This code makes the choloropleth map.'''

    # Store the location codes (also called FIPS codes)
    fips = df_all['location_code'].tolist()
    values = df_all['AvgAQI_' + pollutant].tolist()
    
    # Store the end-points 
    endpts = list(np.linspace(min(values), max(values), len(color_scale) - 1))

    # Create the choloropleth map
    fig = ff.create_choropleth(
        fips = fips, values = values, scope = ['usa'],
        binning_endpoints = endpts, colorscale = color_scale,
        show_state_data = False,
        show_hover = True, centroid_marker = {'opacity': 0},
        asp = 2.9, title = 'USA by Average ' + plot_labels[pollutant]['title'],
        legend_title = 'Avg. ' + plot_labels[pollutant]['title']
    )

    # Show the chloropleth map
    iplot(fig, filename = 'choropleth_full_usa')
    
    return

In [ ]:
# Run the code
if __name__ == '__main__':

    # Store the labels dictionary 
    plot_labels = {'o3': {'title': 'O3'}, 'co': {'title': 'CO'}, 
                   'pm25_frm': {'title': 'PM 2.5'}, 'no2': {'title': 'NO2'}, 
                  'so2': {'title': 'SO2'}} 

    # Store the color-scale
    color_scale = ["#f7fbff","#ebf3fb","#deebf7","#d2e3f3","#c6dbef","#b3d2e9","#9ecae1",
                  "#85bcdb","#6baed6","#57a0ce","#4292c6","#3082be","#2171b5","#1361a9",
                  "#08519c","#0b4083","#08306b"]
    
    # Make the plot for each pollutant
    for pollutant in pollutants:
        make_plot(pollutant, plot_labels, color_scale)

## Train Test Split

In [ ]:
df_all.head()

In [ ]:
categorical_cols = list(df_all.select_dtypes(include=['object']).columns)
print(categorical_cols)

In [ ]:
merge_cols = [col for col in df_all.columns if '_merge' in col]
print(merge_cols)

In [ ]:
df_all.drop(columns=[col for col in df_all.columns if '_merge' in col], inplace=True)

In [ ]:
df_all.to_csv('Alldata.csv')

In [ ]:
categorical_cols = list(df_all.select_dtypes(include=['object']).columns)
print(categorical_cols)

## Convert to Numerical

In [ ]:
df_all.describe()

In [ ]:
df_all.dtypes 

In [ ]:
df=df_all

In [ ]:
df['state_code'] = df['state_code'].astype(float)
df['county_code'] = df['county_code'].astype(float)
df['location_code'] = df['location_code'].astype(float)

In [ ]:
df.dtypes 

In [ ]:
!ls

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Load the data
data = pd.read_csv('Alldata.csv',index_col=0)
print(data.head())
# Convert county column to categorical
data['county'] = pd.Categorical(data['county'])

In [ ]:
data.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Set the year column as the index of the DataFrame
data.set_index('year', inplace=True)

In [ ]:
# Convert the index to a DatetimeIndex
data.index = pd.to_datetime(data.index, format='%Y')

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
# Resample the data by year
df_yearly = data.resample('A').mean()

In [ ]:
# Plot the time series
plt.plot(df_yearly.index, df_yearly['AvgAQI_o3'])
plt.title('Average AQI O3 over time')
plt.xlabel('Year')
plt.ylabel('AQI')
plt.show()

In [ ]:
# Split the data into training and testing sets
train_size = int(len(df_yearly) * 0.8)
train, test = df_yearly[:train_size], df_yearly[train_size:]

In [ ]:
!pip install -q statsmodels==0.10.2

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

model = SARIMAX(train['AvgAQI_o3'], order=(0,1,0), seasonal_order=(0,1,0,12))
model_fit = model.fit()
predictions = model_fit.forecast(steps=len(test))

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(test['AvgAQI_o3'], predictions)
rmse = mean_squared_error(test['AvgAQI_o3'], predictions, squared=False)

print("MAE:", mae)
print("RMSE:", rmse)


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(test.index, test['AvgAQI_o3'], label='Actual')
plt.plot(test.index, predictions, label='Predicted')
plt.legend()
plt.title('SARIMA Model Predictions')
plt.xlabel('Year')
plt.ylabel('AvgAQI_o3')
plt.show()